<h1> Practice project 2 </h1>

- task: compute position of country centers on the map we plotted last time
- hint: check the SVG file via notepad
- SVG image is a file of plain text in XML format
    - XML consists of elements, which consist of attributes
    - in this case: each country is within one <path .../> element
    - also: it has id (path ID) and d (path data) elements
- task has 3 steps:
1) parse the XML in SVG to get d and id attributes for each county
2) extract county's boundary coordinates (as a list of map coordinates) for each county
3) compute the center of each county via provided code and output a CSV file with FIPS codes and those centers
- hint 1: google how to parse xml; task is to write function get_county_attributes(svg_file_name)
- hint 2: extract the coordinates; write function get_boundary_coordinates(boundary_data)
- hint 3: compute county boundaries and output the results to a CSV: function process_country_attributes(svg_file_name, csv_file_name)


In [74]:
"""
Week 2 practice project template for Python Data Visualization
Compute county centers from an SVG image of USA that includes county boundaries
Output a CSV file with FIPS code and county centers
"""
import math
import csv

# Parse the XMLin USA SVG file extract county attributes
# Derive from example code - https://stackoverflow.com/questions/15857818/python-svg-parser
from xml.dom import minidom

def get_county_attributes(svg_file_name):
    """
    Given SVG file associate with string svg_file_name, extract county attributes from associated XML
    Return a list of tuples consisting of FIPS codes (strings) and county boundaries (strings)
    """
    doc = minidom.parse(svg_file_name)
    parsed_svg_list = []
    for i in doc.getElementsByTagName("path"):
        tuple_of_data = (i.getAttribute('id'), i.getAttribute('d'))
        parsed_svg_list.append(tuple_of_data)
    return parsed_svg_list


def get_boundary_coordinates(boundary_data):
    """
    Given the country boundary data as a string,
    Return the county boundary as a list of coordinates,
    where each pair is a tuple of two cordinates.
    These tuples are then items in the list of coordinates.
    Hence returns a list of tuples of x and y coordinates.
    Ignores 'M', 'L, 'z'
    """
    pairs_of_coordinates = []
    boundary_data = boundary_data.split(" ") #split via empty space
    for i in boundary_data:
        if i in ("", "M", "L", "z"): #leaving out unwanted characters
            continue
        else:
            split_pair = i.split(",") #splitting the pair
            pair_list = []
            for x in split_pair:
                pair_list.append(float(x.rstrip("M")))  #making a list of the pair coordinates as floats
            pairs_of_coordinates.append(tuple(pair_list)) #putting this pair to the list as tuple
    return pairs_of_coordinates

def dist(pt1, pt2):
    """
    Compute Euclidean distance between two points
    """
    return math.sqrt((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2)

def compute_county_center(boundary_coordinates):
    """
    Given a list of coordinates (tuples of two floats) on the county boundary,
    Return an estimate of the center of the county as a tuple of two floats (x and y coordinates)
    Assumes the list of coordinates forms a closed polygon with first and last point repeated
    """
    centroid = [0, 0]
    perimeter = 0
    for idx in range(len(boundary_coordinates) - 1):
        edge_length = dist(boundary_coordinates[idx], boundary_coordinates[idx + 1])
        centroid[0] += 0.5 * (boundary_coordinates[idx][0] + boundary_coordinates[idx + 1][0]) * edge_length
        centroid[1] += 0.5 * (boundary_coordinates[idx][1] + boundary_coordinates[idx + 1][1]) * edge_length
        perimeter += edge_length
    return [(centroid[0] / perimeter), (centroid[1] / perimeter)]

def process_county_attributes(svg_file_name, csv_file_name):
    """
    Given SVG file name (as string), extract county attributes (FIPS code and county boundaries)
    Then compute county centers and write a CSV file with columns corresponding to FIPS code, x-coord of centers, y-coord of centers 
    """
    parsed_data = get_county_attributes(svg_file_name)
    #getting list of tuple pairs (id, data)
    with open(csv_file_name, "w", newline="") as csv_file:
        writer = csv.writer(csv_file, delimiter=",")
        for i in parsed_data:   #going through each county pair
            county_id = i[0]    #id is first, hence index 0
            boundary_coordinates = get_boundary_coordinates(i[1])   #coordinates are 2nd
            county_centers = compute_county_center(boundary_coordinates)
            list_to_write = []  #making a list I can write as one line
            list_to_write.append(county_id)
            for i in county_centers:
                list_to_write.append(i)
            writer.writerow(list_to_write) #writing the list

   
process_county_attributes("USA_Counties_2014.svg", "USA_Counties_with_FIPS_and_centers.csv")   
# Output CSV file should have 3143 rows
    #IT DOES!